In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
import ctapipe
from lstchain.io import replace_config, standard_config
import h5py
from astropy.io import fits
import json
muon_rings = [434, 663, 949, 1295, 1355, 1565, 1777, 1933, 3067, 3179, 3270, 3314,3346, 3692, 4076,  5016, 6853, 7723, 7919, 8122, 8605, 9047, 9157, 9239, 9774,  9910, 10486, 10491, 11600, 11781, 12038, 12506, 12969, 13004, 13101] 
muon_rings_datacheck = [54110, 55656, 55942, 57623, 58136, 58798, 59194, 59570, 60407, 60422, 62395, 63033, 63412, 64971, 65272, 66251, 66737, 67143, 67175, 67772, 68483, 68495, 68755, 68902, 69300, 69891, 70855, 72398, 73259, 73261, 73725, 76581, 76871, 77277, 80267, 80982, 81229, 81606, 82140, 83970, 84318, 85580, 87424, 87427, 87921, 88388, 89632, 91351, 93414, 93971, 94651, 95006, 96661, 97072, 97242, 97335, 97713, 97785, 97891, 99847, 100267, 101084, 101156, 101606, 103028, 103475, 105020, 105713]
flagged_rings_sim = [5333507, 1673009, 1808303, 3362305, 2835508, 4026706, 159503, 4854202, 5464504, 2439702, 1402200, 3456509, 3547004, 2229705, 22604, 5573308, 3489206, 417000, 5895006, 1836008, 1794401, 593906, 3280901, 1114103, 5920700, 5521903, 5240406, 5678007, 918003, 5858809, 4315801, 3547006, 729206, 5412204, 759808, 3386907, 2597005, 3539901, 4125904, 1073405, 3662503, 846606, 2845905, 5139408, 5097001, 3248609, 2257305, 5559707, 1775404, 5586609, 5387503, 336801, 3611003, 4504102, 1556201, 3742006, 55203, 377809, 5303909, 3393108, 5854105, 3850800, 5240903, 5120108, 4330605, 4006201, 4752000, 817501, 1096109, 5562603, 3289408, 2057308, 1646106, 2736603, 220203, 2095301, 243205, 250008, 728702, 1177701, 402700, 4589808, 1063003, 3804102, 1088507, 5638409, 1824304, 2482107, 2619809, 4647903, 5864908, 4805401, 2936707, 996201]
len(muon_rings)
import os
filename = '/Users/vdk/simtel_corsika_theta_30.390_az_93.640_run1.simtel.gz'
source = EventSource(filename)
event_iterator = iter(source)

### Observations

In [ ]:
event_waveforms = {}
muon_waveforms_obs = []
#foldername = '/Users/vdk/RealLST/waveforms_low/'
#foldername = '/Users/vdk/RealLST/waveforms_obs14948/'
foldername = '/Users/vdk/RealLST/waveforms/'
#foldername = '/Users/vdk/RealLST/waveforms1subrun/'
for filename in os.listdir(foldername):
    #event_waveforms[filename[10:16]] = np.load('/Users/vdk/UnrealLST/waveforms/' + filename)
    #event_waveforms[filename[10:16]] = np.load('/Users/vdk/RealLST/waveforms/' + filename)
    muon_waveforms_obs.append(np.load(foldername + filename, allow_pickle=True))

str_rings = [str(number) for number in flagged_rings_sim]

flag = True
counter = 0
counter = 0
proton_waveforms_obs = []
#for filename2 in os.listdir('/Users/vdk/UnrealLST/waveformsProton/'):

#foldername = '/Users/vdk/RealLST/waveforms_low_proton/'
foldername = '/Users/vdk/RealLST/waveforms14948run1subrun_proton/'
#foldername = '/Users/vdk/RealLST/waveforms1subrun_protons/'
#foldername = '/Users/vdk/RealLST/waveforms1subrun_lot/'
for filename2 in os.listdir(foldername):
  proton_waveforms_obs.append(np.load(foldername + filename2, allow_pickle=True))
  if counter == 1000:
    break



min_time = 7
max_time = 12

arr_max_muons = []
arg_max_muons = []

arg_max_muons_mask = []


variance_muon = []


muon_pixel_number = []

for waveform in muon_waveforms_obs:
   tmp_var_muon = []
   tmp_pixel = 0
   pixel_mask = np.array(np.max(waveform, axis = 1) > 2.5, dtype = bool)
   for pixel in waveform[pixel_mask]:
      arg_max_muons_mask.append(np.argmax(pixel))
      tmp_pixel += 1
    
   for pixel in waveform[:, min_time:max_time]:
      tmp_var_muon.append(np.var(pixel))
   
   variance_muon.append(tmp_var_muon)
   muon_pixel_number.append(tmp_pixel)


arg_max_protons = []
arg_max_protons_mask = []
variance_proton = []
proton_pixel_number = []
stopper = 0

for waveform in proton_waveforms_obs:
   tmp_var_proton = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2.5, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton.append(tmp_var_proton)
   proton_pixel_number.append(tmp_pixel)
   stopper += 1
   if stopper == 10000:
      break


In [ ]:
np.shape(muon_waveforms_obs[0])

### Simulations

In [ ]:
event_waveforms = {}
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms/'):
    event_waveforms[filename[10:16]] = np.load('/Users/vdk/UnrealLST/waveforms/' + filename)
    #muon_waveforms.append( np.load('/Users/vdk/RealLST/waveforms/' + filename, allow_pickle=True))


str_rings = [str(number) for number in flagged_rings_sim]
proton_waveforms = []
flag = True
counter = 0


counter = 0
proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveformsProton/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveformsProton/' + filename2, allow_pickle=True))
  if counter == 1000:
    break



min_time = 9
max_time = 14

arg_max_muons_mask_sim = []

variance_muon_sim = []


muon_pixel_number_sim = []

for key in event_waveforms:
   tmp_var_muon = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(event_waveforms[key], axis = 1) > 2, dtype = bool)
      for pixel in event_waveforms[key][pixel_mask]:
        arg_max_muons_mask_sim.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in event_waveforms[key][:, min_time:max_time]:
        tmp_var_muon.append(np.var(pixel))
   
   variance_muon_sim.append(tmp_var_muon)
   muon_pixel_number_sim.append(tmp_pixel)



arg_max_protons_mask_sim = []

variance_proton_sim = []

proton_pixel_number_sim = []

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_sim.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_sim.append(tmp_var_proton)
   proton_pixel_number_sim.append(tmp_pixel)


### Variance

In [ ]:
hist_density = False
var_hist_m_short = [np.mean(np.array(item)) for item in variance_muon] # only for considered time span (7:12)
var_hist_p_short = [np.mean(np.array(item)) for item in variance_proton]
var_hist_m_short_sim = [np.mean(np.array(item)) for item in variance_muon_sim] # only for considered time span (7:12)
var_hist_p_short_sim = [np.mean(np.array(item)) for item in variance_proton_sim]
bin_number = 30
plt.figure(figsize = (9,6))
z,x,c = plt.hist(var_hist_p_short, alpha = 0.35, density = hist_density,color = '#1f77b4', bins = bin_number, range =(0,1), label='not muons in data')
plt.hist(var_hist_m_short, alpha = 0.35, density = hist_density, color = 'orange', bins = bin_number, range=(0,1), label='muons in data')

#plt.hist(var_hist_p_short_sim, alpha = 1, density = True, color = 'red', bins = bin_number, range =(0,1), label='not muons in simulation', histtype='step')
#plt.hist(var_hist_m_short_sim, alpha = 1, density = True, bins = bin_number, color = 'green', range=(0,1), label='muons in simulation', histtype='step')

plt.grid(alpha = 0.5)
plt.legend()
plt.ylim(0,100)
#plt.ylim(0,2)
plt.xlim(0,0.6)
plt.xlabel('Mean of variance per pixel for every event')
#plt.ylabel('Probability density')
plt.ylabel('Count')
#plt.title(f'Run{14948} Distribution of mean for variance per pixel for timebins [{min_time}:{max_time}] - data; [9:14] - sim')
plt.title(f'Run{14948} Distribution of mean for variance per pixel for timebins [{min_time}:{max_time}]')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

In [ ]:
sorted(var_hist_m_short, reverse=False)
#len(var_hist_p_short)

### Argmax

In [ ]:
plt.figure(figsize = (9,6))
bin_number1 = 35
bin_number2 = 40
# Time distribution
k, z, l = plt.hist(arg_max_protons_mask, bins = bin_number1, color = '#1f77b4', density = True, alpha = 0.35, label = 'not muons in data')
muons_val, muons_bin, l = plt.hist(arg_max_muons_mask, bins = bin_number1, density = True,color = 'orange', alpha = 0.35, label = 'muons in data')

k, z, l = plt.hist(arg_max_protons_mask_sim, bins = bin_number2, density = True, color = 'red', alpha = 1, label = 'not muons in simulation', histtype = 'step')
k, z, l = plt.hist(arg_max_muons_mask_sim, bins = bin_number2, density = True, color = 'green', alpha = 1, label = 'muons in simulation', histtype = 'step')




plt.legend()
plt.xlabel("Time of max R1 amplitude")
plt.ylabel("Probability density")
plt.grid(alpha = 0.5)
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_timesample_R1cut3.png")


### Pixel number

In [ ]:
protons_outlier = [250 for i in proton_pixel_number if i >= 250]
protons_outlier_sim = [250 for i in proton_pixel_number_sim if i >= 250]

final_proton_number = proton_pixel_number + protons_outlier
final_proton_number_sim = proton_pixel_number_sim + protons_outlier_sim

bin_number = 25
plt.figure(figsize = (9,6))
z,x,c = plt.hist(final_proton_number, alpha = 0.35, density = False,color = '#1f77b4', bins = bin_number, range =(0,250), label='not muons in data')
plt.hist(muon_pixel_number, alpha = 0.35, density = False, color = 'orange', bins = bin_number, range=(0,250), label='muons in data')

#plt.hist(final_proton_number_sim, alpha = 1, density = True, color = 'red', bins = bin_number, range =(0,250), label='not muons in simulation', histtype='step')
#plt.hist(muon_pixel_number_sim, alpha = 1, density = True, bins = bin_number, color = 'green', range=(0,250), label='muons in simulation', histtype='step')

plt.grid(alpha = 0.5)
plt.legend()
plt.ylim(0,50)
plt.xlabel('Number of pixels on the image with R1 amplitude > 2.5')
#plt.ylabel('Probability density')
plt.ylabel('Probability density')
plt.ylabel('Count')
plt.title('Distribution of pixel number for simulation and real data (overflow bin 250)')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

In [ ]:
# in 1st run we have two rings 5405008 and 4431008
filename = '/Users/vdk/simtel_corsika_theta_30.390_az_93.640_run1.simtel.gz'
source = EventSource(filename)
event_iterator = iter(source)
for waveform in muon_waveforms:
    #if 1 == event.trigger.tels_with_trigger[0]:
    # fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
    # fig.subplots_adjust(hspace=0.2)
    # ax = axes[0]
    # camgeom = source.subarray.tel[1].camera.geometry
    # #title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
    # title = '1'
    # disp = CameraDisplay(camgeom,title=title, ax = ax)
    # #disp.image = event.r1.tel[teln].waveform[:,11]
    # #disp.image = event.dl0.tel[teln].waveform[:,11]
    # disp.image = waveform.sum(axis = 1)
    # disp.cmap = plt.cm.RdBu_r
    # disp.add_colorbar()
    # disp.set_limits_percent(95)
    # bx = axes[1]
    # for pix_id in range(0,1855):
    #     #bx.plot(j.r0.tel[teln].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    #     bx.plot(waveform[pix_id], label="pix {}".format(pix_id), drawstyle='steps')
    camgeom = source.subarray.tel[1].camera.geometry
    #title=f"event{iter_number}_fit{round(j.muon.tel[teln].efficiency.optical_efficiency,3)}_width{round(j.muon.tel[teln].efficiency.width.to_value(),3)}"
    title = '1'
    disp = CameraDisplay(camgeom,title=title)
    #disp.image = event.r1.tel[teln].waveform[:,11]
    #disp.image = event.dl0.tel[teln].waveform[:,11]
    disp.image = waveform.sum(axis = 1)
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()
    #if i == 20:
    #   break
plt.close()        

### Optical efficiency scan

In [ ]:
muon_waveforms_0634 = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0634/'):
    muon_waveforms_0634.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0634/' + filename, allow_pickle=True))

proton_waveforms_0634 = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0634/'):
  proton_waveforms_0634.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0634/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0634 = []
variance_muon_0634 = []
muon_pixel_number_0634 = []

for i,waveform in enumerate(muon_waveforms_0634):
  tmp_var_muon = []
  tmp_pixel = 0
  if i not in [10,14,24]:
    pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
    for pixel in waveform[pixel_mask]:
      arg_max_muons_mask_0634.append(np.argmax(pixel))
      tmp_pixel += 1

    for pixel in waveform[:, min_time:max_time]:
      tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0634.append(tmp_var_muon)
  muon_pixel_number_0634.append(tmp_pixel)



arg_max_protons_mask_0634 = []
variance_proton_0634 = []
proton_pixel_number_0634 = []

for waveform in proton_waveforms_0634:
   tmp_var_proton = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0634.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0634.append(tmp_var_proton)
   proton_pixel_number_0634.append(tmp_pixel)


In [ ]:
muon_waveforms_0684 = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0684/'):
    muon_waveforms_0684.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0684/' + filename, allow_pickle=True))

proton_waveforms_0684 = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0684/'):
  proton_waveforms_0684.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0684/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0684 = []
variance_muon_0684 = []
muon_pixel_number_0684 = []

for i,waveform in enumerate(muon_waveforms_0684):
  tmp_var_muon = []
  tmp_pixel = 0
  print(i)
  if not i in [2,5]:
    print(i)
    pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
    for pixel in waveform[pixel_mask]:
      arg_max_muons_mask_0684.append(np.argmax(pixel))
      tmp_pixel += 1
 
  for pixel in waveform[:, min_time:max_time]:
    tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0684.append(tmp_var_muon)
  muon_pixel_number_0684.append(tmp_pixel)



arg_max_protons_mask_0684 = []
variance_proton_0684 = []
proton_pixel_number_0684 = []
counter = 0
for waveform in proton_waveforms_0684:
   tmp_var_proton = []
   tmp_pixel = 0
   counter += 1
   if counter < 300:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0684.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0684.append(tmp_var_proton)
   proton_pixel_number_0684.append(tmp_pixel)

len(arg_max_muons_mask_0684)


In [ ]:
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0734/'):
    muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0734/' + filename, allow_pickle=True))

proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0734/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0734/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0734 = []
variance_muon_0734 = []
muon_pixel_number_0734 = []

for waveform in muon_waveforms:
  tmp_var_muon = []
  tmp_pixel = 0
  pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
  for pixel in waveform[pixel_mask]:
    arg_max_muons_mask_0734.append(np.argmax(pixel))
    tmp_pixel += 1

  for pixel in waveform[:, min_time:max_time]:
    tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0734.append(tmp_var_muon)
  muon_pixel_number_0734.append(tmp_pixel)



arg_max_protons_mask_0734 = []
variance_proton_0734 = []
proton_pixel_number_0734 = []
counter = 0

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   counter += 1
   if counter < 300:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0734.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0734.append(tmp_var_proton)
   proton_pixel_number_0734.append(tmp_pixel)


In [ ]:
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0784/'):
    muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0784/' + filename, allow_pickle=True))

proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0784/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0784/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0784 = []
variance_muon_0784 = []
muon_pixel_number_0784 = []

for waveform in muon_waveforms:
  tmp_var_muon = []
  tmp_pixel = 0
  pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
  for pixel in waveform[pixel_mask]:
    arg_max_muons_mask_0784.append(np.argmax(pixel))
    tmp_pixel += 1

  for pixel in waveform[:, min_time:max_time]:
    tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0784.append(tmp_var_muon)
  muon_pixel_number_0784.append(tmp_pixel)



arg_max_protons_mask_0784 = []
variance_proton_0784 = []
proton_pixel_number_0784 = []
counter = 0
for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   counter += 1
   if counter < 300:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0784.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0784.append(tmp_var_proton)
   proton_pixel_number_0784.append(tmp_pixel)


In [ ]:
for i,waveform in enumerate(muon_waveforms):
    plt.figure(figsize = (18,12))
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'{i}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.highlight_pixels(pixel_mask)
    disp.image = waveform.sum(axis = 1)
    disp.cmap = plt.cm.Reds
    disp.add_colorbar()
    #disp.set_limits_percent(95)
    disp.set_limits_minmax(0,20)
    plt.show()

In [ ]:
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0834/'):
    muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0834/' + filename, allow_pickle=True))

proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0834/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0834/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0834 = []
variance_muon_0834 = []
muon_pixel_number_0834 = []

for waveform in muon_waveforms:
  tmp_var_muon = []
  tmp_pixel = 0
  pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
  for pixel in waveform[pixel_mask]:
    arg_max_muons_mask_0834.append(np.argmax(pixel))
    tmp_pixel += 1

  for pixel in waveform[:, min_time:max_time]:
    tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0834.append(tmp_var_muon)
  muon_pixel_number_0834.append(tmp_pixel)



arg_max_protons_mask_0834 = []
variance_proton_0834 = []
proton_pixel_number_0834 = []
counter = 0

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   counter += 1
   if counter < 300:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0834.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0834.append(tmp_var_proton)
   proton_pixel_number_0834.append(tmp_pixel)


In [ ]:
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0884/'):
    muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0884/' + filename, allow_pickle=True))

proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0884/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0884/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0884 = []
variance_muon_0884 = []
muon_pixel_number_0884 = []

for waveform in muon_waveforms:
  tmp_var_muon = []
  tmp_pixel = 0
  pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
  for pixel in waveform[pixel_mask]:
    arg_max_muons_mask_0884.append(np.argmax(pixel))
    tmp_pixel += 1

  for pixel in waveform[:, min_time:max_time]:
    tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0884.append(tmp_var_muon)
  muon_pixel_number_0884.append(tmp_pixel)



arg_max_protons_mask_0884 = []
variance_proton_0884 = []
proton_pixel_number_0884 = []
counter = 0

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   counter += 1
   if counter < 300:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0884.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0884.append(tmp_var_proton)
   proton_pixel_number_0884.append(tmp_pixel)


In [ ]:
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0934/'):
    muon_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveforms0934/' + filename, allow_pickle=True))

proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0934/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveforms_opt_scan/waveformsProton0934/' + filename2, allow_pickle=True))



min_time = 9
max_time = 14

arg_max_muons_mask_0934 = []
variance_muon_0934 = []
muon_pixel_number_0934 = []

for waveform in muon_waveforms:
  tmp_var_muon = []
  tmp_pixel = 0
  pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
  for pixel in waveform[pixel_mask]:
    arg_max_muons_mask_0934.append(np.argmax(pixel))
    tmp_pixel += 1

  for pixel in waveform[:, min_time:max_time]:
    tmp_var_muon.append(np.var(pixel))
   
  variance_muon_0934.append(tmp_var_muon)
  muon_pixel_number_0934.append(tmp_pixel)



arg_max_protons_mask_0934 = []
variance_proton_0934 = []
proton_pixel_number_0934 = []
counter = 0 

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   counter += 1
   if counter < 300:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_0934.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_0934.append(tmp_var_proton)
   proton_pixel_number_0934.append(tmp_pixel)


### Distributions for opt eff scan

#### Variance

In [ ]:
var_hist_m_short_0634 = [np.mean(np.array(item)) for item in variance_muon_0634] # only for considered time span (9:14)
var_hist_p_short_0634 = [np.mean(np.array(item)) for item in variance_proton_0634]
var_hist_m_short_0684 = [np.mean(np.array(item)) for item in variance_muon_0684] # only for considered time span (9:14)
var_hist_p_short_0684 = [np.mean(np.array(item)) for item in variance_proton_0684]
var_hist_m_short_0734 = [np.mean(np.array(item)) for item in variance_muon_0734] # only for considered time span (9:14)
var_hist_p_short_0734 = [np.mean(np.array(item)) for item in variance_proton_0734]
var_hist_m_short_0784 = [np.mean(np.array(item)) for item in variance_muon_0784] # only for considered time span (9:14)
var_hist_p_short_0784 = [np.mean(np.array(item)) for item in variance_proton_0784]
var_hist_m_short_0834 = [np.mean(np.array(item)) for item in variance_muon_0834] # only for considered time span (9:14)
var_hist_p_short_0834 = [np.mean(np.array(item)) for item in variance_proton_0834]
var_hist_m_short_0884 = [np.mean(np.array(item)) for item in variance_muon_0884] # only for considered time span (9:14)
var_hist_p_short_0884 = [np.mean(np.array(item)) for item in variance_proton_0884]
var_hist_m_short_0934 = [np.mean(np.array(item)) for item in variance_muon_0934] # only for considered time span (9:14)
var_hist_p_short_0934 = [np.mean(np.array(item)) for item in variance_proton_0934]

bin_number = 50
alpha_param = 0.25
plt.figure(figsize = (9,6))

#plt.hist(var_hist_p_short_0634, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.634 reflection')
plt.hist(var_hist_m_short_0634, alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.634 reflection')

#plt.hist(var_hist_p_short_0684, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')
plt.hist(sorted(var_hist_m_short_0684)[2:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')

#plt.hist(var_hist_p_short_0734, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.734 reflection')
plt.hist(sorted(var_hist_m_short_0734)[5:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.734 reflection')

#plt.hist(var_hist_p_short_0784, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.784 reflection')
plt.hist(sorted(var_hist_m_short_0784)[7:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.784 reflection')

#plt.hist(var_hist_p_short_0834, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.834 reflection')
plt.hist(sorted(var_hist_m_short_0834)[8:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.834 reflection')

#plt.hist(var_hist_p_short_0884, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.834 reflection')
plt.hist(sorted(var_hist_m_short_0884)[10:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.834 reflection')

#plt.hist(var_hist_p_short_0934, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.834 reflection')
plt.hist(sorted(var_hist_m_short_0934)[9:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.834 reflection')

#z,x,c = plt.hist(var_hist_p_short, alpha = 1, density = True,color = '#1f77b4', bins = bin_number, range =(0,1), label='protons in observation', histtype='step', lw = 2)
plt.hist(var_hist_m_short, alpha = 1, density = True, color = 'black', bins = bin_number, range=(0,1), label='muons in observation', histtype='step', lw = 2)

plt.grid(alpha = 0.5)
#plt.ylim(0,40)
plt.legend()
plt.xlabel('Mean of variance per pixel for every event')
plt.ylabel('Probability density')
plt.title('Distribution of mean for variance for simulations with different mirror degraded reflection')
#plt.savefig("/Users/vdk/RealLST/images/Simulation_optScan_MuonVariance_R1cut2_9-14timebin.png")

In [ ]:
var_hist_m_short_0634 = [np.mean(np.array(item)) for item in variance_muon_0634] # only for considered time span (9:14)
var_hist_p_short_0634 = [np.mean(np.array(item)) for item in variance_proton_0634]
var_hist_m_short_0684 = [np.mean(np.array(item)) for item in variance_muon_0684] # only for considered time span (9:14)
var_hist_p_short_0684 = [np.mean(np.array(item)) for item in variance_proton_0684]
var_hist_m_short_0734 = [np.mean(np.array(item)) for item in variance_muon_0734] # only for considered time span (9:14)
var_hist_p_short_0734 = [np.mean(np.array(item)) for item in variance_proton_0734]
var_hist_m_short_0784 = [np.mean(np.array(item)) for item in variance_muon_0784] # only for considered time span (9:14)
var_hist_p_short_0784 = [np.mean(np.array(item)) for item in variance_proton_0784]
var_hist_m_short_0834 = [np.mean(np.array(item)) for item in variance_muon_0834] # only for considered time span (9:14)
var_hist_p_short_0834 = [np.mean(np.array(item)) for item in variance_proton_0834]
var_hist_m_short_0884 = [np.mean(np.array(item)) for item in variance_muon_0884] # only for considered time span (9:14)
var_hist_p_short_0884 = [np.mean(np.array(item)) for item in variance_proton_0884]
var_hist_m_short_0934 = [np.mean(np.array(item)) for item in variance_muon_0934] # only for considered time span (9:14)
var_hist_p_short_0934 = [np.mean(np.array(item)) for item in variance_proton_0934]

bin_number = 50
alpha_param = 0.25
plt.figure(figsize = (9,6))

plt.hist(var_hist_p_short_0634, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.634 reflection')
#plt.hist(var_hist_m_short_0634, alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.634 reflection')

plt.hist(var_hist_p_short_0684, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')
#plt.hist(sorted(var_hist_m_short_0684)[2:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')

plt.hist(var_hist_p_short_0734, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.734 reflection')
#plt.hist(sorted(var_hist_m_short_0734)[5:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.734 reflection')

plt.hist(var_hist_p_short_0784, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.784 reflection')
#plt.hist(sorted(var_hist_m_short_0784)[7:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.784 reflection')

plt.hist(var_hist_p_short_0834, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.834 reflection')
#plt.hist(sorted(var_hist_m_short_0834)[8:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.834 reflection')

plt.hist(var_hist_p_short_0884, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.834 reflection')
#plt.hist(sorted(var_hist_m_short_0884)[10:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.834 reflection')

plt.hist(var_hist_p_short_0934, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.834 reflection')
#plt.hist(sorted(var_hist_m_short_0934)[9:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.834 reflection')

#z,x,c = plt.hist(var_hist_p_short, alpha = 1, density = True,color = 'black', bins = bin_number, range =(0,1), label='protons in observation', histtype='step', lw = 2)
#plt.hist(var_hist_m_short, alpha = 1, density = True, color = 'black', bins = bin_number, range=(0,1), label='muons in observation', histtype='step', lw = 2)

plt.grid(alpha = 0.5)
#plt.ylim(0,40)
plt.legend()
plt.xlabel('Mean of variance per pixel for every event')
plt.ylabel('Probability density')
plt.title('Distribution of mean for variance for simulations with different mirror degraded reflection')
#plt.savefig("/Users/vdk/RealLST/images/Simulation_optScan_MuonVariance_R1cut2_9-14timebin.png")

In [ ]:
fig, axs = plt.subplots(4, 2,figsize = (15,12))
plt.grid()
alpha_param = 0.5
bin_number = 40

axs[0, 0].hist(var_hist_m_short_0634, alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.634 reflection')
axs[0, 0].set_title('Simulations with mirror reflection 0.634')
axs[0, 0].hist(var_hist_p_short_0634, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.634 reflection')
#axs[0, 1].set_title('Axis [0, 1]')

axs[0, 1].hist(sorted(var_hist_m_short_0684)[2:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')
axs[0, 1].set_title('Simulations with mirror reflection 0.684')
axs[0, 1].hist(var_hist_p_short_0684, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')
#axs[1, 1].set_title('Axis [1, 1]')

axs[1, 0].hist(sorted(var_hist_m_short_0734)[5:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')
axs[1, 0].set_title('Simulations with mirror reflection 0.734')
axs[1, 0].hist(var_hist_p_short_0734, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')

axs[1, 1].hist(sorted(var_hist_m_short_0784)[7:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')
axs[1, 1].set_title('Simulations with mirror reflection 0.784')
axs[1, 1].hist(var_hist_p_short_0784, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')

axs[2, 0].hist(sorted(var_hist_m_short_0834)[8:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')
axs[2, 0].set_title('Simulations with mirror reflection 0.834')
axs[2, 0].hist(var_hist_p_short_0834, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')

axs[2, 1].hist(sorted(var_hist_m_short_0884)[10:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')
axs[2, 1].set_title('Simulations with mirror reflection 0.884')
axs[2, 1].hist(var_hist_p_short_0834, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')

axs[3, 0].hist(sorted(var_hist_m_short_0934)[10:], alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons with 0.684 reflection')
axs[3, 0].set_title('Simulations with mirror reflection 0.934')
axs[3, 0].hist(var_hist_p_short_0884, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons for 0.684 reflection')

axs[3, 1].hist(var_hist_m_short, alpha = alpha_param, density = True, bins = bin_number, range=(0,1), label='muons in observation')
axs[3, 1].set_title('Observations')
axs[3, 1].hist(var_hist_p_short, alpha = alpha_param, density = True, bins = bin_number, range =(0,1), label='protons in observation')

for ax in axs.flat:
    ax.grid(alpha = 0.5)
    
for ax in axs.flat:
    ax.set(xlabel='mean(variance)', ylabel='Probability Density')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
sorted(var_hist_m_short_0934)

In [ ]:
len(var_hist_m_short_0634)

#### Argmax

In [ ]:

bin_number = 36

alpha_param = 0.25
plt.figure(figsize = (9,6))

plt.hist(arg_max_protons_mask_0634, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.634 reflection')
plt.hist(arg_max_muons_mask_0634, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')

plt.hist(arg_max_protons_mask_0684, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')
plt.hist(arg_max_muons_mask_0684, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.684 reflection')

plt.hist(arg_max_protons_mask_0734, bins = bin_number,  density = True, alpha = 0.35, label = 'protons with 0.734 reflection')
plt.hist(arg_max_muons_mask_0734, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.734 reflection')

plt.hist(arg_max_protons_mask_0784, bins = bin_number,  density = True, alpha = 0.35, label = 'protons with 0.784 reflection')
plt.hist(arg_max_muons_mask_0784, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.784 reflection')

plt.hist(arg_max_protons_mask_0834, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.834 reflection')
plt.hist(arg_max_muons_mask_0834, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.834 reflection')

plt.hist(arg_max_protons_mask_0884, bins = bin_number,  density = True, alpha = 0.35, label = 'protons with 0.884 reflection')
plt.hist(arg_max_muons_mask_0884, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.884 reflection')

plt.hist(arg_max_protons_mask_0934, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.934 reflection')
plt.hist(arg_max_muons_mask_0934, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.934 reflection')

#z,x,c = plt.hist(var_hist_p_short, alpha = 1, density = True,color = '#1f77b4', bins = bin_number, range =(0,1), label='protons in observation', histtype='step', lw = 2)
#plt.hist(var_hist_m_short, alpha = 1, density = True, color = 'black', bins = bin_number, range=(0,1), label='muons in observation', histtype='step', lw = 2)

plt.grid(alpha = 0.5)
#plt.ylim(0.01,1)
plt.legend()
plt.xlabel('Mean of variance per pixel for every event')
plt.ylabel('Probability density')
plt.title('Distribution of mean for variance for simulations with different mirror degraded reflection')
#plt.savefig("/Users/vdk/RealLST/images/Simulation_optScan_MuonVariance_R1cut2_9-14timebin.png")

In [ ]:
fig, axs = plt.subplots(4, 2,figsize = (15,12))
plt.grid()
alpha_param = 0.5
bin_number = 40

axs[0, 0].hist(arg_max_muons_mask_0634, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[0, 0].set_title('Simulations with mirror reflection 0.634')
axs[0, 0].hist(arg_max_protons_mask_0634, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.634 reflection')
#axs[0, 1].set_title('Axis [0, 1]')

axs[0, 1].hist(arg_max_muons_mask_0684, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[0, 1].set_title('Simulations with mirror reflection 0.684')
axs[0, 1].hist(arg_max_protons_mask_0684, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')
#axs[1, 1].set_title('Axis [1, 1]')

axs[1, 0].hist(arg_max_muons_mask_0734, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[1, 0].set_title('Simulations with mirror reflection 0.734')
axs[1, 0].hist(arg_max_protons_mask_0734, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')

axs[1, 1].hist(arg_max_muons_mask_0784, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[1, 1].set_title('Simulations with mirror reflection 0.784')
axs[1, 1].hist(arg_max_protons_mask_0784, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')

axs[2, 0].hist(arg_max_muons_mask_0834, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[2, 0].set_title('Simulations with mirror reflection 0.834')
axs[2, 0].hist(arg_max_protons_mask_0834, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')

axs[2, 1].hist(arg_max_muons_mask_0884, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[2, 1].set_title('Simulations with mirror reflection 0.884')
axs[2, 1].hist(arg_max_protons_mask_0884, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')

axs[3, 0].hist(arg_max_muons_mask_0934, bins = bin_number, density = True, alpha = 0.35, label = 'muons with 0.634 reflection')
axs[3, 0].set_title('Simulations with mirror reflection 0.934')
axs[3, 0].hist(arg_max_protons_mask_0934, bins = bin_number, density = True, alpha = 0.35, label = 'protons with 0.684 reflection')

axs[3, 1].hist(arg_max_muons_mask, bins = bin_number-4, density = True, alpha = 0.35, label = 'muons in observation')
axs[3, 1].set_title('Observations')
axs[3, 1].hist(arg_max_protons_mask, bins = bin_number-4, density = True, alpha = 0.35, label = 'protons in observation')

for ax in axs.flat:
    ax.grid(alpha = 0.5)
    
for ax in axs.flat:
    ax.set(xlabel='time sample of waveform peak', ylabel='Probability Density')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

#### Number pixels

In [ ]:
protons_outlier = [250 for i in proton_pixel_number if i >= 250]
protons_outlier_0634 = [250 for i in proton_pixel_number_0634 if i >= 250]
protons_outlier_0684 = [250 for i in proton_pixel_number_0684 if i >= 250]
protons_outlier_0734 = [250 for i in proton_pixel_number_0734 if i >= 250]
protons_outlier_0784 = [250 for i in proton_pixel_number_0784 if i >= 250]
protons_outlier_0834 = [250 for i in proton_pixel_number_0834 if i >= 250]
protons_outlier_0884 = [250 for i in proton_pixel_number_0884 if i >= 250]
protons_outlier_0934 = [250 for i in proton_pixel_number_0934 if i >= 250]

final_proton_number = proton_pixel_number + protons_outlier
final_proton_number_0634 = proton_pixel_number_0634 + protons_outlier_0634
final_proton_number_0684 = proton_pixel_number_0684 + protons_outlier_0684
final_proton_number_0734 = proton_pixel_number_0734 + protons_outlier_0734
final_proton_number_0784 = proton_pixel_number_0784 + protons_outlier_0784
final_proton_number_0834 = proton_pixel_number_0834 + protons_outlier_0834
final_proton_number_0884 = proton_pixel_number_0884 + protons_outlier_0884
final_proton_number_0934 = proton_pixel_number_0934 + protons_outlier_0934

muon_pixel_number_0634 = sorted(muon_pixel_number_0634.copy())[2:]
muon_pixel_number_0684 = sorted(muon_pixel_number_0684.copy())[2:]
muon_pixel_number_0734 = sorted(muon_pixel_number_0734.copy())[5:]
muon_pixel_number_0784 = sorted(muon_pixel_number_0784.copy())[7:]
muon_pixel_number_0834 = sorted(muon_pixel_number_0834.copy())[8:]
muon_pixel_number_0884 = sorted(muon_pixel_number_0884.copy())[10:]
muon_pixel_number_0934 = sorted(muon_pixel_number_0934.copy())[9:]



#plt.legend()
#plt.xlabel('Number of pixels on the image with R1 amplitude > 2.5 p.e./ns')
#plt.ylabel('Probability density')
#plt.title('Distribution of pixel number for simulation and real data (overflow bin 250)')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

fig, axs = plt.subplots(4, 2,figsize = (15,12))
plt.grid(alpha = 0.5)
alpha_param = 0.5
bin_number = 25

axs[0, 0].hist(muon_pixel_number_0634, alpha = 1, density = True, bins = bin_number, range=(0,250), label='muons in simulation')
axs[0, 0].set_title('Simulations with mirror reflection 0.634')
axs[0, 0].hist(final_proton_number_0634, alpha = 1, density = True,  bins = bin_number, range =(0,250), label='protons in simulation')
#axs[0, 1].set_title('Axis [0, 1]')

axs[0, 1].hist(muon_pixel_number_0684, alpha = 1, density = True, bins = bin_number,  range=(0,250), label='muons in simulation')
axs[0, 1].set_title('Simulations with mirror reflection 0.684')
axs[0, 1].hist(final_proton_number_0684, alpha = 1, density = True,  bins = bin_number, range =(0,250), label='protons in simulation')
#axs[1, 1].set_title('Axis [1, 1]')

axs[1, 0].hist(muon_pixel_number_0734, alpha = 1, density = True, bins = bin_number,  range=(0,250), label='muons in simulation')
axs[1, 0].set_title('Simulations with mirror reflection 0.734')
axs[1, 0].hist(final_proton_number_0734, alpha = 1, density = True,  bins = bin_number, range =(0,250), label='protons in simulation')

axs[1, 1].hist(muon_pixel_number_0784, alpha = 1, density = True, bins = bin_number,range=(0,250), label='muons in simulation')
axs[1, 1].set_title('Simulations with mirror reflection 0.784')
axs[1, 1].hist(final_proton_number_0784, alpha = 1, density = True,  bins = bin_number, range =(0,250), label='protons in simulation')

axs[2, 0].hist(muon_pixel_number_0834, alpha = 1, density = True, bins = bin_number,  range=(0,250), label='muons in simulation')
axs[2, 0].set_title('Simulations with mirror reflection 0.834')
axs[2, 0].hist(final_proton_number_0834, alpha = 1, density = True,  bins = bin_number, range =(0,250), label='protons in simulation')

axs[2, 1].hist(muon_pixel_number_0884, alpha = 1, density = True, bins = bin_number,  range=(0,250), label='muons in simulation')
axs[2, 1].set_title('Simulations with mirror reflection 0.884')
axs[2, 1].hist(final_proton_number_0884, alpha = 1, density = True,  bins = bin_number, range =(0,250), label='protons in simulation')

axs[3, 0].hist(muon_pixel_number_0934, alpha = 1, density = True, bins = bin_number,  range=(0,250), label='muons in simulation')
axs[3, 0].set_title('Simulations with mirror reflection 0.934')
axs[3, 0].hist(final_proton_number_0934, alpha = 1, density = True, bins = bin_number, range =(0,250), label='protons in simulation')

axs[3, 1].hist(muon_pixel_number, alpha = 0.35, density = True, color = 'orange', bins = bin_number, range=(0,250), label='muons in observation')
axs[3, 1].set_title('Observations')
axs[3, 1].hist(final_proton_number, alpha = 0.35, density = True,color = '#1f77b4', bins = bin_number, range =(0,250), label='protons in observation')

for ax in axs.flat:
    ax.grid(alpha = 0.5)
    
for ax in axs.flat:
    ax.set(xlabel='Number of pixels on the image with R1 amplitude > 2.5 p.e./ns', ylabel='Probability Density')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

### Graphs

#### Simulation R1 waveforms

In [ ]:
for i,waveform in enumerate(proton_waveforms):
    plt.figure(figsize = (18,12))
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'{i}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.highlight_pixels(pixel_mask)
    disp.image = waveform.sum(axis = 1)
    disp.cmap = plt.cm.Reds
    disp.add_colorbar()
    #disp.set_limits_percent(95)
    disp.set_limits_minmax(0,20)
    plt.show()

#### Observational waveforms

In [ ]:
for i,waveform in enumerate(proton_waveforms_obs):
    plt.figure(figsize = (18,12))
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'{i}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.highlight_pixels(pixel_mask)
    disp.image = waveform.sum(axis = 1)
    disp.cmap = plt.cm.Reds
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #disp.set_limits_minmax(0,20)
    plt.show()

In [ ]:
for i,waveform in enumerate(muon_waveforms_obs):
    plt.figure(figsize = (18,12))
    plt.grid()
    for pix_id in range(0, 1855):
        plt.plot(waveform[pix_id],drawstyle="steps")

    plt.show()

In [ ]:
waveform[1,:].shape